# 📰 Détection de Fake News avec un Modèle Simple
Ce notebook propose un pipeline complet pour détecter automatiquement les fake news à partir de textes d'articles.

## 📥 1. Importation des bibliothèques

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


## 📄 2. Chargement des données

In [ ]:
# Téléchargez le dataset manuellement sur Kaggle si besoin
# Dataset : https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset
df_fake = pd.read_csv("Fake.csv")
df_real = pd.read_csv("True.csv")

df_fake["label"] = 0  # Fake
df_real["label"] = 1  # Real

df = pd.concat([df_fake, df_real]).sample(frac=1).reset_index(drop=True)
df = df[["text", "label"]]
df.head()


## 🧹 3. Prétraitement et vectorisation

In [ ]:
X = df["text"]
y = df["label"]

vectorizer = TfidfVectorizer(stop_words="english", max_df=0.7)
X_vec = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)


## 🤖 4. Entraînement du modèle

In [ ]:
model = MultinomialNB()
model.fit(X_train, y_train)


## 📊 5. Évaluation du modèle

In [ ]:
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Fake", "Real"], yticklabels=["Fake", "Real"])
plt.xlabel("Prédit")
plt.ylabel("Réel")
plt.title("Matrice de confusion")
plt.show()


## 🚀 6. Interface utilisateur avec Gradio (optionnel)

In [ ]:
import gradio as gr

def predict_news(text):
    vec = vectorizer.transform([text])
    pred = model.predict(vec)[0]
    return "🟢 VRAI" if pred == 1 else "🔴 FAUX"

interface = gr.Interface(fn=predict_news, inputs="textbox", outputs="text", title="Détecteur de Fake News")
interface.launch()
